In [59]:
import numpy as np
import os
import cv2

In [216]:
from conv_new2 import convolution

In [288]:
path = '/Users/rajeshr/Desktop/Group_22/selected/'

images = []
color_images = []
label = []
for i in os.listdir(path):
    image = cv2.imread(path+i, cv2.IMREAD_GRAYSCALE)
    color_image = cv2.imread(path+i, cv2.COLOR_BGR2RGB)
    try:
        images.append(cv2.resize(image, (224, 224)))
        color_images.append(cv2.resize(color_image, (224, 224)))
        label.append(i)
    except:
        pass
    
images = np.array(images)
label = np.array(label)
color_images = np.array(color_images)
labels = np.arange(0, len(label))

In [224]:
class kaiming_initialization:
    
    def initialize(self, n, size):
        mean, sd = 0, np.sqrt(2/n)
        weights = np.random.normal(mean, sd, size=size)
        return weights

In [226]:
class ConvolutionLayer:
    
    def forward_prop(self, image, K, size, n):
        output = []
        ReLUout = []
        filters = []
        for i in range(K):
            filtr = kaiming_initialization.initialize(kaiming_initialization, n, size)
            filters.append(filtr)
            res = convolution.convolve(convolution, inpt=image, filtr=filtr, stride = 1, padding = 0)
            output.append(res)
            ReLUout.append(ReLUActivation.forward_prop(ReLUActivation, res))
            
        self.output = np.array(output)
        self.ReLUout = np.array(ReLUout)
        self.filters = np.array(filters)
        
    def inputlayerConV(self, image, K, size, n):
        output = []
        ReLUout = []
        filters = []
        for i in range(K):
            filtr = kaiming_initialization.initialize(kaiming_initialization, n, size)
            filters.append(filtr)
            res = convolution.inptconvolve(convolution, inpt=image, filtr=filtr, stride = 1, padding = 0)
            output.append(res)
            ReLUout.append(ReLUActivation.forward_prop(ReLUActivation, res))
            
        self.output = np.array(output)
        self.ReLUout = np.array(ReLUout)
        self.filters = np.array(filters)

In [227]:
class MaxPool:
    
    def forward_prop(self, featureMaps, size, stride):
        m, n = featureMaps[0].shape
        fm, fn = size
        result = []
        poolout = []
        for featureMap in featureMaps:
            for i in range(0, m, stride):
                for j in range(0, n, stride):
                    if featureMap[i:i+fm, j:j+fn].shape == size:
                        result.append(max(featureMap[i:i+fm, j:j+fn].flatten()))
                        
            Outm, Outn = int(((m-fm)/stride)+1), int(((n-fn)/stride)+1)
            poolout.append(np.array(result).reshape((Outm, Outn)))
            result = []
        
        self.maxpoolout = np.array(poolout)

In [265]:
class ReLUActivation:
    
    def forward_prop(self, x):
        
        out = []
        if len(x.shape) == 1:
            for i in x:
                if i>=0:
                    out.append(i)
                else:
                    out.append(0)
        else:
            for i in x:
                for j in i:
                    if j>=0:
                        out.append(j)
                    else:
                        out.append(0)
                        
        return np.array(out).reshape(x.shape)

In [229]:
class SoftmaxActivation:
    
    def forward_prop(self, x):
        x = np.array(x, dtype=np.float64)
        exp = np.exp(x)
        return exp/np.sum(exp)

In [349]:
class Deepneuralnetwork:
    
    def forward_prop(self, xn, wh):
        self.xn = xn
        self.wh = wh
        h1 = np.dot(wh.T, xn)
        self.output = ReLUActivation.forward_prop(ReLUActivation, h1)

    def Outforward_prop(self, xn, wo):
        self.Oxn = xn
        self.wo = wo
        out = np.dot(wo.T, xn)
        self.foutput = SoftmaxActivation.forward_prop(SoftmaxActivation, out)
        
    def flattenBackprop(self, dy):
        if dy.shape[0] == self.xn.shape[0]:
            dy = dy.T
        dw = dy.dot(self.xn)
        dx = np.dot(dy.T, self.wh.T)
        self.wh -= 0.001 * dw.T
        return dx
        
    def backprop(self, dy):
        if dy.shape[0] == self.output.shape[0]:
            dy = dy.T
        dw = dy.dot(self.output)
        dx = np.dot(dy.T, self.wh.T)
        self.wh -= 0.001 * dw.T
        return dx
    
    def Outbackprop(self, dy):
        if dy.shape[0] == self.foutput.shape[0]:
            dy = dy.T
        dw = dy.dot(self.foutput)
        dx = np.dot(dy.T, self.wo.T)
        self.wo -= 0.001 * dw.T
        return dx

In [350]:
class Flattening:
    
    def flatten(self, x):
        self.flatten_out = x.flatten()

In [351]:
class loss:
    
    def cross_entropy(self, inputs, labels):

        out_num = labels.shape[0]
        p = np.sum(labels.reshape(1,out_num)*inputs)
        loss = -np.log(p)
        return loss

In [352]:
layer1 = ConvolutionLayer()
layer2 = MaxPool()
layer3 = ConvolutionLayer()
layer4 = MaxPool()
layer5 = Flattening()
layer6 = Deepneuralnetwork()
layer7 = Deepneuralnetwork()
kaiming = kaiming_initialization()
losses = loss()

In [ ]:
epoch = 25

for image in tr_data:
    for i in range(epoch):
        layer1.inputlayerConV(image, 32, (3, 3, 3), 27)
        layer2.forward_prop(layer1.ReLUout, (2, 2), 1)
        layer3.forward_prop(layer2.maxpoolout, 64, (32, 3, 3), 27)
        layer4.forward_prop(layer3.ReLUout, (2, 2), 1)
        layer5.flatten(layer4.maxpoolout)

        wh = kaiming.initialize(layer5.flatten_out.shape[0]*128, (layer5.flatten_out.shape[0], 128))
        wo = kaiming.initialize(128, (128, 3))
        layer6.forward_prop(layer5.flatten_out, wh)
        layer7.Outforward_prop(layer6.output, wo)
        
        acc = 0
        total_acc = 0
        labell = np.argmax(layer7.foutput)
        entropyloss = losses.cross_entropy(layer7.foutput, labels)
        if np.argmax(layer7.foutput) == np.argmax(labels):
            acc += 1
            total_acc += 1

        dy = labels
        
        b_layer7 = layer7.Outbackprop(dy)
        b_layer6 = layer6.backprop(b_layer7)
        b_layer4 = layer4.backprop(b_layer6)
        b_layer3 = layer3.backprop(b_layer3)
        b_layer2 = layer2.backprop(b_layer2)
        b_layer1 = layer1.backprop(b_layer1)
        
        print("Epoch: {} LOSS: {}, Accuracy: {}".format(i+1, entropyloss, total_acc))
        
print("DATA TRAINED!!")

In [ ]:
def test_model(image):
    layer1.inputlayerConV(image, 32, (3, 3, 3), 27)
    layer2.forward_prop(layer1.ReLUout, (2, 2), 1)
    layer3.forward_prop(layer2.maxpoolout, 64, (32, 3, 3), 27)
    layer4.forward_prop(layer3.ReLUout, (2, 2), 1)
    layer5.flatten(layer4.maxpoolout)

    wh = kaiming.initialize(layer5.flatten_out.shape[0]*128, (layer5.flatten_out.shape[0], 128))
    wo = kaiming.initialize(128, (128, 3))
    layer6.forward_prop(layer5.flatten_out, wh)
    layer7.Outforward_prop(layer6.output, wo)

    acc = 0
    total_acc = 0
    labell = np.argmax(layer7.foutput)
    predicted = label1
    entropyloss = losses.cross_entropy(layer7.foutput, labels)
    if np.argmax(layer7.foutput) == np.argmax(labels):
        acc += 1
        total_acc += 1
        
    print("Epoch: {} Test LOSS: {}, Test Accuracy: {}".format(i+1, entropyloss, total_acc))
    
    return predicted

In [ ]:
predicted = test_model(test_data)

In [ ]:
cm = tf.math.confusion_matrix(test_label, p_class)
cm

In [ ]:
model = [layer1, layer2, layer3, layer4, layer5, layer6, layer7]

In [ ]:
layer = model.layers #Conv layers at 0, 2
filters, biases = model.layers[2].get_weights()
print(layer[2].name, filters.shape)

In [ ]:
for i in range(64):
    f = filters[:, :, :, i]
    plt.imshow(f[:, :, 0], cmap='gray')
    plt.savefig('/Users/rajeshr/Desktop/f/filters'+str(i+1)+'.png')

In [ ]:
conv_layer_index = [0] #[0, 1]
outputs = [model.layers[i].output for i in conv_layer_index]
model_short = Model(inputs=model.inputs, outputs=outputs)
print(model_short.summary())

In [ ]:
img = load_img('/Users/rajeshr/Desktop/Group_22/resized/chandelier.jpg', target_size=(224, 224))
img = img_to_array(img)
img = np.expand_dims(img, axis=0)
feature_output = model_short.predict(img)

In [ ]:
for ftr in feature_output:
    for i in range(32):
        plt.imshow(ftr[:, :, i], cmap='gray') #ftr[0, :, :, i]
        plt.savefig('/Users/rajeshr/Desktop/fm/fm'+str(i+1)+'.png')